In [1]:
import os.path as osp

import torch
from sklearn.metrics import roc_auc_score

import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv
from torch_geometric.utils import negative_sampling

if torch.cuda.is_available():
    device = torch.device('cuda')
elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import torch.nn as nn
import torch.nn.functional as F
import numpy as np


#https://colab.research.google.com/drive/1N3LvAO0AXV4kBPbTMX866OwJM9YS6Ji2?usp=sharing#scrollTo=fl5W1gg5Jhzz

df_players = pd.read_csv("./../data_scrapped/atp_players.csv")
df_matchs = pd.read_csv("./../data_formatted/training_dgl_dataset.csv")

In [3]:
#On construit le graph

# lecture des données attendues
# tensor1[1] -> tensor2[1] 
# le player index en tensor1 a joué contre le player au meme index en tensor 2
# y stocke le résultat du match


tensor1=[]
tensor2=[]

for index,row in df_players.iterrows():
    winmatchs = df_matchs[df_matchs.player1_id == row.player_id]
    #print(len(winmatchs))
    if len(winmatchs) > 0:
        #print(row.player_id)
        for index2, row2 in winmatchs.iterrows():
                tensor1.append(index)
                tensor2.append(df_players.loc[df_players.player_id == row2.player2_id].index[0])

In [4]:
import numpy
import random

labels = numpy.ones(len(tensor1))

indexes = random.sample(range(0, len(tensor1)-1), ((len(tensor1)-1)//2))

for index in indexes:
    temp = tensor1[index]
    tensor1[index] = tensor2[index]
    tensor2[index] = temp
    labels[index] = 0

In [10]:
import torch_geometric.transforms as T
from torch_geometric.data import Data

# Create the heterogeneous graph data object:
#data = Data()

# Add the user nodes:
x = torch.tensor(list(df_players[["birth_year","weight_kg","height_cm"]].values),dtype=torch.float)  # [num_users, num_features_users]
x = torch.nan_to_num(x, nan=0.0)
#x = torch.masked_select(x, ~torch.isnan(x))
#x = torch.ones(df_players.shape[0])

edge_index = torch.stack([torch.tensor(tensor1), torch.tensor(tensor2)], dim=0)
edge_attr = torch.Tensor(list(df_matchs[["player1_atprank","player1_oddsB365","player2_atprank","player2_oddsB365"]].values))
#y = torch.Tensor(list(df_matchs[["winner_player1"]].values))

labels = torch.tensor(labels, dtype=torch.long)

data = Data(x=x, edge_index=edge_index,edge_label=labels) 
#data = T.ToUndirected()(data)
# Add the movie nodes:
#data['movie'].x = movie_features  # [num_movies, num_features_movies]

data

C:\Users\chbou\AppData\Local\Temp\ipykernel_27432\1253515148.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels, dtype=torch.long)


Data(x=[3446, 3], edge_index=[2, 51981], edge_label=[51981])

In [6]:
labels

tensor([0, 1, 1,  ..., 1, 0, 1])

In [11]:
train_data, val_data, test_data = T.RandomLinkSplit(
    num_val=0.05,
    num_test=0.1,
    is_undirected=False,
    add_negative_train_samples=False,
    neg_sampling_ratio=0
)(data)
train_data, val_data, test_data

(Data(x=[3446, 3], edge_index=[2, 44184], edge_label=[44184], edge_label_index=[2, 44184]),
 Data(x=[3446, 3], edge_index=[2, 44184], edge_label=[2599], edge_label_index=[2, 2599]),
 Data(x=[3446, 3], edge_index=[2, 46783], edge_label=[5198], edge_label_index=[2, 5198]))

In [20]:
from torch_geometric.nn import GATConv,MLP
from torch_geometric.loader import DataLoader
from tqdm import tqdm

class Net(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GATConv(3, 128, edge_dim = 4)
        self.conv2 = GATConv(128, 64, edge_dim = 4)
        self.mlp = MLP(in_channels=128, hidden_channels=128, out_channels=1, num_layers=3)

    def forward(self, x, edge_index, edge_label_index, edge_attr):
        x = self.conv1(x, edge_index, edge_attr).relu()
        x = self.conv2(x, edge_index, edge_attr).relu()
        x = self.conv3(x, edge_index, edge_attr).relu()
        return self.mlp(torch.cat((x[edge_label_index[0]], x[edge_label_index[1]]), dim=-1))
    
model = Net().to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()


def train(data):
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index, data.edge_label_index, data.edge_attr)
    loss = criterion(out.squeeze(), data.edge_label.float())
    loss.backward()
    optimizer.step()
    return loss

@torch.no_grad()
def test(data):
    model.eval()
    out = model(data.x, data.edge_index, data.edge_label_index, data.edge_attr)
    return roc_auc_score(data.edge_label.float(), out.squeeze())


best_val_auc = final_test_auc = 0
for epoch in range(1, 101):
    loss = train(train_data)
    val_auc = test(val_data)
    test_auc = test(test_data)
    if val_auc > best_val_auc:
        best_val_auc = val_auc
        final_test_auc = test_auc
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Val: {val_auc:.4f}, 'f'Test: {test_auc:.4f}')


Epoch: 001, Loss: 0.7113, Val: 0.4923, Test: 0.4938
Epoch: 002, Loss: 0.7020, Val: 0.4863, Test: 0.4940
Epoch: 003, Loss: 0.7203, Val: 0.4902, Test: 0.4996
Epoch: 004, Loss: 0.6930, Val: 0.5007, Test: 0.5074
Epoch: 005, Loss: 0.6969, Val: 0.5234, Test: 0.5119
Epoch: 006, Loss: 0.7048, Val: 0.5063, Test: 0.5179
Epoch: 007, Loss: 0.6985, Val: 0.5107, Test: 0.5187
Epoch: 008, Loss: 0.6982, Val: 0.5041, Test: 0.5088
Epoch: 009, Loss: 0.6963, Val: 0.5205, Test: 0.4946
Epoch: 010, Loss: 0.6936, Val: 0.5171, Test: 0.5107
Epoch: 011, Loss: 0.6909, Val: 0.5136, Test: 0.4919
Epoch: 012, Loss: 0.6924, Val: 0.4936, Test: 0.4970
Epoch: 013, Loss: 0.6916, Val: 0.4880, Test: 0.5083
Epoch: 014, Loss: 0.6909, Val: 0.5035, Test: 0.5003
Epoch: 015, Loss: 0.6910, Val: 0.5097, Test: 0.5292
Epoch: 016, Loss: 0.6909, Val: 0.5063, Test: 0.5212
Epoch: 017, Loss: 0.6908, Val: 0.5182, Test: 0.4938
Epoch: 018, Loss: 0.6907, Val: 0.4985, Test: 0.5142
Epoch: 019, Loss: 0.6907, Val: 0.5175, Test: 0.5215
Epoch: 020, 

In [18]:
# Specify the edge you want to predict (e.g., edge from node 0 to node 1)
node_0 = 3
node_1 = 664

print(df_players.iloc[node_0])
#print(df_players.iloc[node_0]["first_name","last_name"])

# Predict the direction of the edge
with torch.no_grad():
    model.eval()
    edge_label_index =  torch.tensor([[node_0, node_1], [node_1, node_0]], dtype=torch.long).t()
    print(edge_label_index)
    prediction = model(data.x, data.edge_index, edge_label_index, data.edge_attr).view(-1).sigmoid()
    print(prediction)

player_id                l836
first_name               Nick
last_name             Lindahl
country_code              AUS
birthdate          1988.07.31
birth_year             1988.0
birth_month               7.0
birth_day                31.0
turned_pro             2006.0
weight_kg                77.0
weight_lbs              170.0
height_cm               183.0
height_in                72.0
birthplace      Malmo, Sweden
Name: 3, dtype: object
tensor([[  3, 664],
        [664,   3]])
tensor([0.3646, 0.6394])
